In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical
import re


Using TensorFlow backend.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
data = pd.read_csv('Sheet_1.csv')
# Keeping only the neccessary columns
data = data[['class','response_text']]


In [4]:
print(data[ data['class'] == 'flagged'].size)
print(data[ data['class'] == 'not_flagged'].size)

50
110


In [5]:
max_fatures = 2000
tokenizer = Tokenizer(nb_words=max_fatures, split=' ')
tokenizer.fit_on_texts(data['class'].values)
X = tokenizer.texts_to_sequences(data['class'].values)
X = pad_sequences(X)



/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/preprocessing/text.py:145: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


In [6]:
embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(max_fatures, embed_dim,input_length = X.shape[1], dropout=0.2))
model.add(LSTM(lstm_out, dropout_U=0.2, dropout_W=0.2))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())


/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:5: UserWarning: The `dropout` argument is no longer support in `Embedding`. You can apply a `keras.layers.SpatialDropout1D` layer right after the `Embedding` layer to get the same behavior.
/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `LSTM` call to the Keras 2 API: `LSTM(196, dropout=0.2, recurrent_dropout=0.2)`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 2, 128)            256000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 196)               254800    
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 394       
Total params: 511,194
Trainable params: 511,194
Non-trainable params: 0
_________________________________________________________________
None


In [12]:
Y = pd.get_dummies(data['class']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.33, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)



(53, 2) (53, 2)
(27, 2) (27, 2)


In [9]:
batch_size = 32
model.fit(X_train, Y_train, nb_epoch = 50, batch_size=batch_size, verbose = 1)

Epoch 1/50
53/53 [==============================] - 0s 347us/step - loss: 0.4986 - acc: 0.6604
Epoch 2/50
53/53 [==============================] - 0s 303us/step - loss: 0.4488 - acc: 0.8302
Epoch 3/50
53/53 [==============================] - 0s 286us/step - loss: 0.3938 - acc: 0.9623
Epoch 4/50
53/53 [==============================] - 0s 296us/step - loss: 0.3342 - acc: 1.0000
Epoch 5/50
53/53 [==============================] - 0s 271us/step - loss: 0.2698 - acc: 1.0000
Epoch 6/50
53/53 [==============================] - 0s 265us/step - loss: 0.2063 - acc: 1.0000
Epoch 7/50
53/53 [==============================] - 0s 289us/step - loss: 0.1530 - acc: 1.0000
Epoch 8/50
53/53 [==============================] - 0s 258us/step - loss: 0.1032 - acc: 1.0000
Epoch 9/50
53/53 [==============================] - 0s 275us/step - loss: 0.0701 - acc: 1.0000
Epoch 10/50
53/53 [==============================] - 0s 237us/step - loss: 0.0415 - acc: 1.0000
Epoch 11/50
53/53 [==============================

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Epoch 14/50
53/53 [==============================] - 0s 317us/step - loss: 0.0076 - acc: 1.0000
Epoch 15/50
53/53 [==============================] - 0s 280us/step - loss: 0.0044 - acc: 1.0000
Epoch 16/50
53/53 [==============================] - 0s 269us/step - loss: 0.0038 - acc: 1.0000
Epoch 17/50
53/53 [==============================] - 0s 255us/step - loss: 0.0022 - acc: 1.0000
Epoch 18/50
53/53 [==============================] - 0s 260us/step - loss: 0.0018 - acc: 1.0000
Epoch 19/50
53/53 [==============================] - 0s 281us/step - loss: 0.0013 - acc: 1.0000
Epoch 20/50
53/53 [==============================] - 0s 249us/step - loss: 0.0011 - acc: 1.0000
Epoch 21/50
53/53 [==============================] - 0s 243us/step - loss: 8.6063e-04 - acc: 1.0000
Epoch 22/50
53/53 [==============================] - 0s 268us/step - loss: 7.1827e-04 - acc: 1.0000
Epoch 23/50
53/53 [==============================] - 0s 237us/step - loss: 6.5050e-04 - acc: 1.0000
Epoch 24/50
53/53 [=========

In [10]:
validation_size = 1500

X_validate = X_test[-validation_size:]
Y_validate = Y_test[-validation_size:]
X_test = X_test[:-validation_size]
Y_test = Y_test[:-validation_size]
score,acc = model.evaluate(X_test, Y_test, verbose = 2, batch_size = batch_size)
print("score: %.2f" % (score))
print("acc: %.2f" % (acc))

ValueError: not enough values to unpack (expected 2, got 0)

In [35]:
pred=model.predict_classes(X_test)

In [36]:
from sklearn.metrics import accuracy_score
acc=accuracy_score(pred,Y_test[:,1])
print(acc)

1.0


In [34]:
print(Y_test[:,1])

[1 1 0 1 1 0 1 1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 1 1 1 1 1]


## AutoEncoders

In [37]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)

In [38]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

17465344/17464789 [==============================] - 7s 0us/step


In [39]:
max_words = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [40]:
# create the model
model = Sequential()
model.add(Embedding(top_words, 32, input_length=max_words))
model.add(Flatten())
model.add(Dense(250, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 500, 32)           160000    
_________________________________________________________________
flatten_1 (Flatten)          (None, 16000)             0         
_________________________________________________________________
dense_2 (Dense)              (None, 250)               4000250   
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 251       
Total params: 4,160,501
Trainable params: 4,160,501
Non-trainable params: 0
_________________________________________________________________
None


In [41]:
# Fit the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=2, batch_size=128, verbose=2)
# Final evaluation of the model
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Train on 25000 samples, validate on 25000 samples
Epoch 1/2
 - 13s - loss: 0.5111 - acc: 0.7087 - val_loss: 0.3404 - val_acc: 0.8509
Epoch 2/2
 - 13s - loss: 0.1927 - acc: 0.9266 - val_loss: 0.3006 - val_acc: 0.8740
Accuracy: 87.40%


In [42]:
print(y_train)

[1 0 0 ..., 0 1 0]
